In [ ]:
# Set some environment variables
import os

gpu_num = 0  # GPU to be used. Use "" to use the CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress some TF warnings
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

# Import Sionna
try:
    import sionna
except ImportError:
    # Install Sionna if package is not already installed
    os.system("pip install sionna")
    import sionna

# Configure GPU
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
import warnings

tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore')

# Fix the seed for reproducible results
tf.random.set_seed(42)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

from sionna.phy.channel import AWGN
from sionna.phy.fec.conv import ConvEncoder, ViterbiDecoder
from sionna.phy.fec.ldpc import LDPC5GEncoder, LDPC5GDecoder
from sionna.phy.fec.polar import Polar5GEncoder, Polar5GDecoder, PolarEncoder, PolarSCLDecoder
from sionna.phy.fec.turbo import TurboEncoder, TurboDecoder
from sionna.phy.fec.polar.utils import generate_rm_code
from sionna.phy.mapping import Constellation, Mapper, Demapper, BinarySource
from sionna.phy.utils import compute_ber, ebnodb2no, PlotBER


In [ ]:
BATCH_SIZE = 100
NUM_UT = 1
NUM_BS = 1
NUM_UT_ANT = 1
NUM_BS_ANT = 1

NUM_STREAMS_PER_TX = NUM_UT_ANT

NUM_BITS_PER_SYMBOL = 4
CODERATE = 1

# Number of coded bits
n = int(1000)
print(n)
# Number of information bits
k = int(n * CODERATE)
print(k)


In [ ]:
awgn_channel = AWGN()
binary_source = BinarySource()

# Create a QAM constellation with the desired bits per symbol
constellation = Constellation("qam", num_bits_per_symbol=NUM_BITS_PER_SYMBOL)

# The mapper maps blocks of information bits to constellation symbols
mapper = Mapper(constellation=constellation)

# The demapper produces hard decisions for all coded bits
demapper = Demapper("app", constellation=constellation, hard_out=True)


In [ ]:
def generate_awgn(no, x):
    '''Generates complex AWGN with variance `no`, matching `x`'s shape and dtype.'''
    stddev = tf.sqrt(no / 2.0)
    real_part = tf.random.normal(tf.shape(x), stddev=stddev, dtype=tf.float32)
    imag_part = tf.random.normal(tf.shape(x), stddev=stddev, dtype=tf.float32)
    return tf.complex(real_part, imag_part)


def generate_impulsive_noise(no, x, mixture_prob=0.6, var1_scale=3.4, var2_scale=2.6):
    '''Generates complex Gaussian mixture noise with two components.'''
    shape = tf.shape(x)
    var1 = var1_scale * no
    var2 = var2_scale * no
    stddev1 = tf.sqrt(var1 / 2.0)
    stddev2 = tf.sqrt(var2 / 2.0)

    mask = tf.random.uniform(shape, minval=0.0, maxval=1.0) < mixture_prob

    real_part = tf.where(
        mask,
        tf.random.normal(shape, stddev=stddev1, dtype=tf.float32),
        tf.random.normal(shape, stddev=stddev2, dtype=tf.float32),
    )
    imag_part = tf.where(
        mask,
        tf.random.normal(shape, stddev=stddev1, dtype=tf.float32),
        tf.random.normal(shape, stddev=stddev2, dtype=tf.float32),
    )

    return tf.complex(real_part, imag_part)


In [ ]:
# Define the range of SNRs
snr_range_db = np.arange(-3, 20)  # Range from -3 dB to 19 dB

# Placeholder to store BER values
ber_values = []
ber_values_impulsive = []

for ebno_db in snr_range_db:
    no = ebnodb2no(
        ebno_db,
        num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
        coderate=CODERATE,
    )

    bits = binary_source([BATCH_SIZE, NUM_UT, NUM_STREAMS_PER_TX, k])
    x = mapper(bits)

    # AWGN channel
    y_awgn = awgn_channel(x, no)
    bits_hat = demapper(y_awgn, no)
    ber_values.append(compute_ber(bits, bits_hat).numpy())

    # Impulsive channel
    noise = generate_impulsive_noise(no, x)
    y_impulsive = x + 2.5 * noise
    bits_hat_imp = demapper(y_impulsive, no)
    ber_values_impulsive.append(compute_ber(bits, bits_hat_imp).numpy())

# Calculate theoretical BER for AWGN
snr_linear = 10 ** (snr_range_db / 10)
ber_theory = stats.norm.sf(np.sqrt(2 * snr_linear))

# Plot BER curves
plt.figure()
plt.semilogy(snr_range_db, ber_values, marker='o', label='Simulated AWGN')
plt.semilogy(snr_range_db, ber_values_impulsive, marker='o', label='Simulated Impulsive')
plt.semilogy(snr_range_db, ber_theory, linestyle='--', label='Theoretical AWGN')
plt.xlabel('SNR (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.title('Bit Error Rate vs. Signal-to-Noise Ratio (SNR)')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
class SystemModel(tf.keras.Model):
    def __init__(self, k, num_bits_per_symbol, noise_mode="awgn", impulsive_scale=2.5):
        super().__init__()

        self.k = k
        self.num_bits_per_symbol = num_bits_per_symbol
        self.noise_mode = noise_mode
        self.impulsive_scale = impulsive_scale

        self.source = BinarySource()
        self.constellation = Constellation("qam", num_bits_per_symbol=self.num_bits_per_symbol)
        self.mapper = Mapper(constellation=self.constellation)
        self.demapper = Demapper("app", constellation=self.constellation)

    def _apply_noise(self, x, no):
        if self.noise_mode == "impulsive":
            noise = generate_impulsive_noise(no, x)
            return x + self.impulsive_scale * noise
        return x + generate_awgn(no, x)

    @tf.function()
    def call(self, batch_size, ebno_db):
        no = ebnodb2no(
            ebno_db,
            num_bits_per_symbol=self.num_bits_per_symbol,
            coderate=1,
        )

        u = self.source([batch_size, 1, 1, self.k])
        x = self.mapper(u)
        y = self._apply_noise(x, no)
        u_hat = self.demapper(y, no)

        return u, u_hat


In [ ]:
EBN0_DB_MIN = -3.0  # Minimum value of Eb/N0 [dB] for simulations
EBN0_DB_MAX = 25  # Maximum value of Eb/N0 [dB] for simulations

ber_plots = PlotBER("Channel")

model_awgn = SystemModel(
    k=64,
    num_bits_per_symbol=4,
    noise_mode="awgn",
)

ber_plots.simulate(
    model_awgn,
    ebno_dbs=np.arange(EBN0_DB_MIN, EBN0_DB_MAX + 1, 1),
    batch_size=BATCH_SIZE,
    num_target_block_errors=100,
    legend="AWGN and 16QAM",
    soft_estimates=True,
    max_mc_iter=50,
    show_fig=False,
)

model_impulsive = SystemModel(
    k=64,
    num_bits_per_symbol=4,
    noise_mode="impulsive",
)

ber_plots.simulate(
    model_impulsive,
    ebno_dbs=np.arange(EBN0_DB_MIN, EBN0_DB_MAX + 1, 1),
    batch_size=BATCH_SIZE,
    num_target_block_errors=100,
    legend="Impulsive noise and 16QAM",
    soft_estimates=True,
    max_mc_iter=50,
    show_fig=True,
)


In [ ]:
class SystemModelCoding(tf.keras.Model):
    def __init__(
        self,
        k,
        n,
        num_bits_per_symbol,
        encoder,
        decoder,
        demapping_method="app",
        sim_esno=False,
        cw_estimates=False,
        noise_mode="impulsive",
        impulsive_scale=2.5,
    ):
        super().__init__()

        self.k = k
        self.n = n
        self.sim_esno = sim_esno
        self.cw_estimates = cw_estimates
        self.num_bits_per_symbol = num_bits_per_symbol
        self.noise_mode = noise_mode
        self.impulsive_scale = impulsive_scale

        self.source = BinarySource()
        self.constellation = Constellation("qam", num_bits_per_symbol=self.num_bits_per_symbol)
        self.mapper = Mapper(constellation=self.constellation)
        self.demapper = Demapper(demapping_method, constellation=self.constellation)
        self.encoder = encoder
        self.decoder = decoder

    def _apply_noise(self, x, no):
        if self.noise_mode == "impulsive":
            noise = generate_impulsive_noise(no, x)
            return x + self.impulsive_scale * noise
        return x + generate_awgn(no, x)

    @tf.function()
    def call(self, batch_size, ebno_db):
        if self.sim_esno:
            no = ebnodb2no(ebno_db, num_bits_per_symbol=1, coderate=1)
        else:
            no = ebnodb2no(
                ebno_db,
                num_bits_per_symbol=self.num_bits_per_symbol,
                coderate=self.k / self.n,
            )

        u = self.source([batch_size, NUM_UT, NUM_STREAMS_PER_TX, self.k])
        c = self.encoder(u)
        x = self.mapper(c)
        y = self._apply_noise(x, no)
        llr_ch = self.demapper(y, no)
        u_hat = self.decoder(llr_ch)

        if self.cw_estimates:
            return c, u_hat
        return u, u_hat


In [ ]:
# code parameters
k = 64  # number of information bits per codeword
n = 128  # desired codeword length

# Create list of encoder/decoder pairs to be analyzed.
codes_under_test = []

# 5G LDPC codes with 20 BP iterations
enc = LDPC5GEncoder(k=k, n=n)
dec = LDPC5GDecoder(enc, num_iter=20)
name = "5G LDPC BP-20"
codes_under_test.append([enc, dec, name])

# Polar Codes (SC decoding)
enc = Polar5GEncoder(k=k, n=n)
dec = Polar5GDecoder(enc, dec_type="SC")
name = "5G Polar+CRC SC"
codes_under_test.append([enc, dec, name])

# Polar Codes (SCL decoding) with list size 8.
enc = Polar5GEncoder(k=k, n=n)
dec = Polar5GDecoder(enc, dec_type="SCL", list_size=8)
name = "5G Polar+CRC SCL-8"
codes_under_test.append([enc, dec, name])

# RM codes with SCL decoding
f, _, _, _, _ = generate_rm_code(3, 7)  # equals k=64 and n=128 code
enc = PolarEncoder(f, n)
dec = PolarSCLDecoder(f, n, list_size=8)
name = "Reed Muller (RM) SCL-8"
codes_under_test.append([enc, dec, name])

# Conv. code with Viterbi decoding
enc = ConvEncoder(rate=1 / 2, constraint_length=8)
dec = ViterbiDecoder(gen_poly=enc.gen_poly, method="soft_llr")
name = "Conv. Code Viterbi (constraint length 8)"
codes_under_test.append([enc, dec, name])

# Turbo. codes
enc = TurboEncoder(rate=1 / 2, constraint_length=4, terminate=False)
dec = TurboDecoder(enc, num_iter=8)
name = "Turbo Code (constraint length 4)"
codes_under_test.append([enc, dec, name])


In [ ]:
# run ber simulations for each code we have added to the list
for code in codes_under_test:
    print("\nRunning: " + code[2])

    model = SystemModelCoding(
        k=k,
        n=n,
        num_bits_per_symbol=4,
        encoder=code[0],
        decoder=code[1],
        noise_mode="impulsive",
    )

    ber_plots.simulate(
        model,
        ebno_dbs=np.arange(EBN0_DB_MIN, EBN0_DB_MAX + 1, 1),
        legend=code[2],
        max_mc_iter=10,
        num_target_block_errors=100,
        batch_size=BATCH_SIZE,
        soft_estimates=False,
        early_stop=True,
        show_fig=False,
        add_bler=True,
        forward_keyboard_interrupt=True,
    )

# and show the figure
ber_plots(ylim=(1e-5, 1), show_bler=False)
